In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install -q keras

In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

In [4]:
import os
os.chdir("/content/drive/My Drive/deep learning")
!ls

AlexNet.ipynb
dataAugumentationCNN.ipynb
mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
SQUEEZENET.gdoc
TEST_DIR
TRAIN_DIR
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [7]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#Augumentation with only zoom and grayscale format

train_datagen2 = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip = True,
                             zoom_range = 0.3,
                             rescale=1/255.0
                            )

#Corresponding generator
train_generator2 = train_datagen2.flow_from_directory(
    train,
    color_mode='grayscale',
    target_size = (128,128),
    batch_size=32,
    class_mode = 'categorical',
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen2 = ImageDataGenerator(
     rescale=1/255.0 )
#Using testing data as validation data

validation_generator2 = test_datagen2.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42 )

x_train,y_train = next(train_generator2)
x_test,y_test = next(validation_generator2)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 1)
(32, 128, 128, 1)


In [5]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint_filepath = 'checkpoint/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu', input_shape = (128, 128, 1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
#for fitting the model
from keras.callbacks import EarlyStopping

stop = EarlyStopping(
    monitor = 'val_accuracy',
    mode='max',
    patience= 8,
    verbose = 2,
    restore_best_weights = True
)

STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS,
callbacks=stop
)

Epoch 1/20
12/12 [==============================] - 2s 124ms/step - loss: 15.1484 - accuracy: 0.3683 - val_loss: 3.5927 - val_accuracy: 0.4141
Epoch 2/20
12/12 [==============================] - 1s 111ms/step - loss: 2.1622 - accuracy: 0.4982 - val_loss: 0.7724 - val_accuracy: 0.6953
Epoch 3/20
12/12 [==============================] - 1s 109ms/step - loss: 0.8407 - accuracy: 0.6944 - val_loss: 1.1627 - val_accuracy: 0.6484
Epoch 4/20
12/12 [==============================] - 1s 109ms/step - loss: 0.5938 - accuracy: 0.8036 - val_loss: 0.7230 - val_accuracy: 0.6797
Epoch 5/20
12/12 [==============================] - 1s 112ms/step - loss: 0.5775 - accuracy: 0.7540 - val_loss: 1.0008 - val_accuracy: 0.5391
Epoch 6/20
12/12 [==============================] - 1s 109ms/step - loss: 0.6865 - accuracy: 0.7150 - val_loss: 0.9835 - val_accuracy: 0.6562
Epoch 7/20
12/12 [==============================] - 1s 110ms/step - loss: 0.4888 - accuracy: 0.8195 - val_loss: 0.8184 - val_accuracy: 0.6797
Epoch

**Starting With ResNet50**

In [ ]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD, Adam

In [ ]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (128,128,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation= 'softmax')(x)
resnetmodel = Model(inputs = base_model.input, outputs = predictions)
adam = Adam(lr=0.001)
resnetmodel.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
EPOCH = 30
stopping_resNet = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5,
    verbose = 2,
    restore_best_weights = True
)

restNetModel = resnetmodel.fit(
    train_generator2,
    steps_per_epoch=STEPS,
    epochs=EPOCH,
    validation_data=validation_generator2,
    validation_steps=VAL_STEPS,
)

Epoch 1/30
12/12 [==============================] - 8s 232ms/step - loss: 3.2617 - accuracy: 0.3613 - val_loss: 1.2390 - val_accuracy: 0.4062
Epoch 2/30
12/12 [==============================] - 3s 239ms/step - loss: 1.5639 - accuracy: 0.5127 - val_loss: 1.6913 - val_accuracy: 0.2969
Epoch 3/30
12/12 [==============================] - 2s 147ms/step - loss: 1.0851 - accuracy: 0.6682 - val_loss: 1.9980 - val_accuracy: 0.2969
Epoch 4/30
12/12 [==============================] - 2s 145ms/step - loss: 0.9776 - accuracy: 0.7508 - val_loss: 2.0320 - val_accuracy: 0.4062
Epoch 5/30
12/12 [==============================] - 2s 149ms/step - loss: 0.6591 - accuracy: 0.8074 - val_loss: 5.7230 - val_accuracy: 0.3906
Epoch 6/30
12/12 [==============================] - 2s 147ms/step - loss: 0.3551 - accuracy: 0.8946 - val_loss: 2.1311 - val_accuracy: 0.5000
Epoch 7/30
12/12 [==============================] - 2s 143ms/step - loss: 0.1952 - accuracy: 0.9414 - val_loss: 3.7533 - val_accuracy: 0.3750
Epoch 

Simple CNN with **1 Conv Layer**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.normalization import BatchNormalization
import numpy as np

In [ ]:
model = Sequential()

#1-conv layer
model.add(Conv2D(64, (3,3), padding = 'same', input_shape=(128, 128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#Flatten
model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(3,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 128, 128, 64)      640       
_________________________________________________________________
batch_normalization_18 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 262144)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [ ]:
STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS
)

Epoch 1/20
12/12 [==============================] - 2s 131ms/step - loss: 1.3254 - accuracy: 0.5129 - val_loss: 1.0701 - val_accuracy: 0.3672
Epoch 2/20
12/12 [==============================] - 1s 108ms/step - loss: 0.8331 - accuracy: 0.6489 - val_loss: 0.9589 - val_accuracy: 0.4375
Epoch 3/20
12/12 [==============================] - 1s 112ms/step - loss: 0.7594 - accuracy: 0.6762 - val_loss: 0.9597 - val_accuracy: 0.4453
Epoch 4/20
12/12 [==============================] - 1s 115ms/step - loss: 0.5704 - accuracy: 0.7529 - val_loss: 1.1785 - val_accuracy: 0.3750
Epoch 5/20
12/12 [==============================] - 1s 110ms/step - loss: 0.4760 - accuracy: 0.7913 - val_loss: 1.3317 - val_accuracy: 0.3906
Epoch 6/20
12/12 [==============================] - 1s 112ms/step - loss: 0.5381 - accuracy: 0.7742 - val_loss: 1.5346 - val_accuracy: 0.3828
Epoch 7/20
12/12 [==============================] - 1s 110ms/step - loss: 0.3994 - accuracy: 0.8592 - val_loss: 1.6974 - val_accuracy: 0.4062
Epoch 

In [ ]:
model2 = Sequential()

#1-conv layer
model2.add(Conv2D(64, (3,3), padding = 'same', input_shape=(256, 256,1)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))


#2-conv layer
model2.add(Conv2D(128,(5,5),padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))



#Flatten
model2.add(Flatten())


model2.add(Dense(256))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

model2.add(Dense(512))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))


#output layer
model2.add(Dense(3,activation='softmax'))

opt = Adam(lr = 0.0001)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_4 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_3 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 128)     204928    
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 128, 128)    

In [ ]:
results_3layer = model2.fit(
    x=train_generator2,
    steps_per_epoch = STEPS,
    epochs = EPOCHS,
    validation_data = validation_generator2,
    validation_steps = VAL_STEPS,
)

Epoch 1/20
12/12 [==============================] - 7s 379ms/step - loss: 1.1749 - accuracy: 0.5337 - val_loss: 1.0674 - val_accuracy: 0.4531
Epoch 2/20
12/12 [==============================] - 3s 268ms/step - loss: 0.4747 - accuracy: 0.8105 - val_loss: 1.1214 - val_accuracy: 0.5859
Epoch 3/20
12/12 [==============================] - 3s 269ms/step - loss: 0.4453 - accuracy: 0.8358 - val_loss: 1.2079 - val_accuracy: 0.3828
Epoch 4/20
12/12 [==============================] - 3s 268ms/step - loss: 0.4358 - accuracy: 0.8254 - val_loss: 1.2925 - val_accuracy: 0.3750
Epoch 5/20
12/12 [==============================] - 3s 265ms/step - loss: 0.3741 - accuracy: 0.8945 - val_loss: 1.3577 - val_accuracy: 0.3906
Epoch 6/20
12/12 [==============================] - 3s 265ms/step - loss: 0.2807 - accuracy: 0.9093 - val_loss: 1.4508 - val_accuracy: 0.3984
Epoch 7/20
12/12 [==============================] - 3s 265ms/step - loss: 0.3316 - accuracy: 0.8994 - val_loss: 1.5790 - val_accuracy: 0.4219
Epoch 

**VGG16**

In [8]:
#For RGB images

train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             rescale=1/255.0,
                            )

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
     rescale=1/255.0 )
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)


In [9]:
from keras.applications.vgg16 import VGG16
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

vgg16_weight_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = VGG16(
    weights=vgg16_weight_path,
    include_top=False, 
    input_shape=(128,128,3,)
)

NUM_CLASSES = 3

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.layers[0].trainable = False

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 24579     
Total params: 14,739,267
Trainable params: 24,579
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
EPOCHS_VGG = 30
STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

VGG_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=10,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_VGG,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[VGG_STOP]
)

Epoch 1/30
12/12 [==============================] - 170s 12s/step - loss: 1.2774 - accuracy: 0.4260 - val_loss: 0.8831 - val_accuracy: 0.4609
Epoch 2/30
12/12 [==============================] - 3s 221ms/step - loss: 0.7070 - accuracy: 0.6763 - val_loss: 0.6716 - val_accuracy: 0.8203
Epoch 3/30
12/12 [==============================] - 3s 218ms/step - loss: 0.5615 - accuracy: 0.7778 - val_loss: 0.5255 - val_accuracy: 0.8047
Epoch 4/30
12/12 [==============================] - 3s 220ms/step - loss: 0.4042 - accuracy: 0.8498 - val_loss: 0.5608 - val_accuracy: 0.8125
Epoch 5/30
12/12 [==============================] - 3s 218ms/step - loss: 0.3351 - accuracy: 0.8830 - val_loss: 0.5819 - val_accuracy: 0.7344
Epoch 6/30
12/12 [==============================] - 3s 219ms/step - loss: 0.3158 - accuracy: 0.9063 - val_loss: 0.4966 - val_accuracy: 0.8203
Epoch 7/30
12/12 [==============================] - 3s 222ms/step - loss: 0.3014 - accuracy: 0.8887 - val_loss: 0.6217 - val_accuracy: 0.7578
Epoch 

Mobile Net **V2**

In [12]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop

In [ ]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#For RGB images
train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             preprocessing_function = preprocess_input)

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)


In [13]:
mobilenet_weight_path = 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

mobileNet = MobileNetV2(
    input_shape = (128,128,3),
    alpha=1.0,
    include_top=False,
    weights= mobilenet_weight_path
)

MN_model = Sequential()
MN_model.add(mobileNet)
MN_model.add(layers.GlobalAveragePooling2D())
MN_model.add(layers.Dropout(0.5))
MN_model.add(layers.Dense(3, activation='softmax'))
    
MN_model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.00002),
        metrics=['accuracy']
    )

MN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 2,227,715
Non-trainable params: 34,112
_________________________________________________________________


In [14]:
#for fitting the model
from keras.callbacks import EarlyStopping

EPOCHS_MN = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

MN_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = MN_model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_MN,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[MN_STOP]
)

Epoch 1/30
12/12 [==============================] - 9s 326ms/step - loss: 1.5364 - accuracy: 0.3875 - val_loss: 1.4101 - val_accuracy: 0.3047
Epoch 2/30
12/12 [==============================] - 3s 235ms/step - loss: 1.3141 - accuracy: 0.4101 - val_loss: 1.3256 - val_accuracy: 0.3828
Epoch 3/30
12/12 [==============================] - 3s 235ms/step - loss: 1.0472 - accuracy: 0.5365 - val_loss: 1.2471 - val_accuracy: 0.4766
Epoch 4/30
12/12 [==============================] - 3s 235ms/step - loss: 0.8460 - accuracy: 0.6222 - val_loss: 1.2577 - val_accuracy: 0.5312
Epoch 5/30
12/12 [==============================] - 3s 233ms/step - loss: 0.8391 - accuracy: 0.6481 - val_loss: 1.2282 - val_accuracy: 0.5312
Epoch 6/30
12/12 [==============================] - 3s 231ms/step - loss: 0.5908 - accuracy: 0.7266 - val_loss: 1.1362 - val_accuracy: 0.5938
Epoch 7/30
12/12 [==============================] - 3s 236ms/step - loss: 0.5566 - accuracy: 0.7699 - val_loss: 1.2319 - val_accuracy: 0.5625
Epoch 

**SQUEEZENET**

In [15]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [23]:
bnmomemtum=0.9
def fire(x, squeeze, expand):
  y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
  y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
  y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
  y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
  y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
  return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

x = tf.keras.layers.Input(shape=[128,128, 3]) # input is 192x192 pixels RGB

y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(3, activation='softmax')(y)

model = tf.keras.Model(x, y)
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 128, 128, 32) 896         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 128, 128, 32) 128         conv2d_32[0][0]                  
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 128, 128, 24) 792         batch_normalization_32[0][0]     
____________________________________________________________________________________________

In [22]:
x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

(32, 128, 128, 3)
(32, 128, 128, 3)


In [24]:
#for fitting the model
from keras.callbacks import EarlyStopping
EPOCHS_SQ = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

SQ_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_SQ,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[SQ_STOP]
)

Epoch 1/30
12/12 [==============================] - 5s 280ms/step - loss: 1.0489 - accuracy: 0.3947 - val_loss: 1.2642 - val_accuracy: 0.3906
Epoch 2/30
12/12 [==============================] - 3s 227ms/step - loss: 0.6911 - accuracy: 0.7671 - val_loss: 2.1961 - val_accuracy: 0.4062
Epoch 3/30
12/12 [==============================] - 3s 229ms/step - loss: 0.3871 - accuracy: 0.9216 - val_loss: 1.9829 - val_accuracy: 0.4219
Epoch 4/30
12/12 [==============================] - 3s 231ms/step - loss: 0.3042 - accuracy: 0.9353 - val_loss: 1.0387 - val_accuracy: 0.5703
Epoch 5/30
12/12 [==============================] - 3s 229ms/step - loss: 0.2209 - accuracy: 0.9422 - val_loss: 0.3830 - val_accuracy: 0.8594
Epoch 6/30
12/12 [==============================] - 3s 228ms/step - loss: 0.1631 - accuracy: 0.9732 - val_loss: 0.4312 - val_accuracy: 0.8984
Epoch 7/30
12/12 [==============================] - 3s 232ms/step - loss: 0.1453 - accuracy: 0.9809 - val_loss: 0.2296 - val_accuracy: 0.9453
Epoch 

In [25]:
model.save('sq_model.h5')